In [ ]:
## This is an example of how to use customtkinter with a webcam stream.

import tkinter as tk # pip install tk
import customtkinter # pip install customtkinter
import PIL.Image, PIL.ImageTk # pip install pillow
import cv2 # pip install opencv-python
from pygrabber.dshow_graph import FilterGraph # pip install pygrabber

customtkinter.set_appearance_mode("System")  # Modes: "System" (standard), "Dark", "Light"
customtkinter.set_default_color_theme("blue")  # Themes: "blue" (standard), "green", "dark-blue"

class App(customtkinter.CTk):
    def __init__(self):
        super().__init__()

        # configure window
        self.title("Webcam with customtkinter and TK.Canvas")
        self.geometry('1100x580')

        # configure grid layout (4x4)
        self.grid_columnconfigure(1, weight=1)
        self.grid_columnconfigure((2, 3), weight=0)
        self.grid_rowconfigure((0, 1, 2), weight=1)

        # create sidebar frame with widgets
        self.sidebar_frame = customtkinter.CTkFrame(self, width=350, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=4, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(4, weight=1)
        self.logo_label = customtkinter.CTkLabel(self.sidebar_frame, text="Video Source", font=customtkinter.CTkFont(size=20, weight="bold"))
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))

        #get the available video devices
        self.graph = FilterGraph()
 
        # fill combobox with video devices
        self.combobox = customtkinter.CTkOptionMenu(self.sidebar_frame, values=self.graph.get_input_devices(), command=self.optionmenu_callback)
        self.combobox.grid(row=1, column=0, padx=20, pady=(20, 10))

        # connect button
        self.sidebar_button_1 = customtkinter.CTkButton(self.sidebar_frame, text="Connect Camera", command=self.sidebar_button_event)
        self.sidebar_button_1.grid(row=2, column=0, padx=20, pady=10)


    def optionmenu_callback(self, choice):
        print("optionmenu dropdown clicked:", choice)

    def sidebar_button_event(self):
        print("try to open camera: " + self.combobox.get())   

        for i, device in enumerate(self.graph.get_input_devices() ):   
            if device == self.combobox.get():
                self.video_source = i

        # main window
        self.vid = MyVideoCapture(self.video_source)

        # Create a canvas that can fit the above video source size
        self.canvas = tk.Canvas(self, width = self.vid.width, height = self.vid.height)
        self.canvas.grid(row=0, rowspan=4, column=1)

        self.delay = 15
        self.update()        



    def update(self):
        # Get a frame from the video source
        return_value, frame = self.vid.get_frame()        

        if return_value:
            try:
                #frame = self.analyzeFrame(frame) <-- this is where you would put your image processing code, see webcam_qr.py
                self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(frame))
                self.canvas.create_image(0, 0, image = self.photo, anchor = tk.NW)

            except BaseException:
                    import sys
                    print(sys.exc_info()[0])
                    import traceback
                    print(traceback.format_exc())                
            finally:
                pass    

        self.after(self.delay, self.update)


class MyVideoCapture:
    def __init__(self, video_source=0):
        # Open the video source
        self.vid = cv2.VideoCapture(video_source)
        if not self.vid.isOpened():
            raise ValueError("Unable to open video source", video_source)

        # Get video source width and height
        self.width = self.vid.get(cv2.CAP_PROP_FRAME_WIDTH)
        self.height = self.vid.get(cv2.CAP_PROP_FRAME_HEIGHT)

    def get_frame(self):
        if not self.vid.isOpened():
            return (return_value, None)

        return_value, frame = self.vid.read()
        if return_value:
            # Return a boolean success flag and the current frame converted to BGR
            return (return_value, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        else:
            return (return_value, None)

    # Release the video source when the object is destroyed
    def __del__(self):
        if self.vid.isOpened():
            self.vid.release()


if __name__ == "__main__":
    app = App()
    app.mainloop()

In [ ]:
## This is a simpler example of the webcam application.

from tkinter import *
import cv2
from PIL import Image, ImageTk

# Define a video capture object
vid = cv2.VideoCapture(0)

# Declare the width and height in variables
width, height = 800, 600

# Set the width and height
vid.set(cv2.CAP_PROP_FRAME_WIDTH, width)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# Create a GUI app
app = Tk()

# Bind the app with Escape keyboard to
# quit app whenever pressed
app.bind('<Escape>', lambda e: app.quit())

# Create a label and display it on app
label_widget = Label(app)
label_widget.pack()

# Create a function to open camera and
# display it in the label_widget on app


def open_camera():

	# Capture the video frame by frame
	_, frame = vid.read()

	# Convert image from one color space to other
	opencv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)

	# Capture the latest frame and transform to image
	captured_image = Image.fromarray(opencv_image)

	# Convert captured image to photoimage
	photo_image = ImageTk.PhotoImage(image=captured_image)

	# Displaying photoimage in the label
	label_widget.photo_image = photo_image

	# Configure image in the label
	label_widget.configure(image=photo_image)

	# Repeat the same process after every 10 seconds
	label_widget.after(10, open_camera)


# Create a button to open the camera in GUI app
button1 = Button(app, text="Open Camera", command=open_camera)
button1.pack()

# Create an infinite loop for displaying app on screen
app.mainloop()


In [ ]:
## This is a serial port GUI application using Tkinter and PySerial
# GUI design
import tkinter as tk
from tkinter import scrolledtext

# Communication with serial port
import serial
from serial.tools import list_ports

# Multi-threading
import threading

# Get path
import os

# Use realpath if you want the real path (symlinks are resolved)
# file_path = os.path.realpath(__file__)
FILE_PATH = os.path.abspath("__file__")
ICON_PATH = os.path.join(os.path.dirname("__file__"), "icon.png")


class GUI:
    # GUI main class
    def __init__(self, title):

        self.portNamesList = []
        self.baudRatesList = [
            1200,
            2400,
            4800,
            9600,
            19200,
            38400,
            57600,
            115200,
            230400,
            460800,
            576000,
            921600,
        ]
        self.isAnyPortAvailable = False
        self.isStarted = False
        self.serialPortName = None
        self.serialPortBaud = 9600

        self.serialPortManager = SerialPortManager(self.serialPortBaud)
        self.get_available_serial_ports()

        self.guiUpdateInterval = 40

        self.window = tk.Tk()
        # Title of application window
        self.window.title(title)
        # Icon of application window
        self.window.iconphoto(False, tk.PhotoImage(file=ICON_PATH))

        self.topFrame = tk.Frame(self.window, bg="#cccccc")

        self.scanButton = tk.Button(
            self.topFrame,
            text="Scan Serial Ports",
            bg="#0051ff",
            fg="#ffffff",
            border=0,
            highlightbackground="#ffffff",
            highlightthickness=2,
            activebackground="#1f7cff",
            activeforeground="#ffffff",
            font=("Sans", "10", "bold"),
            command=self.scan_button_command,
        )

        # Define a tk.StringVar for storing selected item in OptionMenu
        self.selectedPort = tk.StringVar()
        # Set default value of selectedPort
        if self.isAnyPortAvailable == False:
            self.portNamesList = ["No ports available"]
        self.selectedPort.set(self.portNamesList[0])

        self.portsOptionMenu = tk.OptionMenu(
            self.topFrame, self.selectedPort, *self.portNamesList
        )

        self.portsOptionMenu.configure(
            bg="#ffffff",
            fg="#222222",
            border=0,
            highlightbackground="#aaaaaa",
            activebackground="#eeeeee",
            activeforeground="#111111",
            direction="left",
            font=("Sans", "10", "bold"),
        )
        if self.isAnyPortAvailable == False:
            self.portsOptionMenu.configure(state="disabled")

        # Define a tk.IntVar for storing selected item in OptionMenu
        self.selectedBaudRate = tk.IntVar()
        # Set default value of selectedBaudRate
        self.selectedBaudRate.set(self.baudRatesList[3])
        self.baudRatesOptionMenu = tk.OptionMenu(
            self.topFrame, self.selectedBaudRate, *self.baudRatesList
        )

        self.baudRatesOptionMenu.configure(
            bg="#ffffff",
            fg="#222222",
            border=0,
            highlightbackground="#aaaaaa",
            activebackground="#eeeeee",
            activeforeground="#111111",
            direction="left",
            font=("Sans", "10", "bold"),
        )
        if self.isAnyPortAvailable == False:
            self.baudRatesOptionMenu.configure(state="disabled")

        self.connectButton = tk.Button(
            self.topFrame,
            text="Connect",
            bg="#00a832",
            fg="#ffffff",
            border=0,
            highlightbackground="#ffffff",
            highlightthickness=2,
            activebackground="#3fcc69",
            activeforeground="#ffffff",
            font=("Sans", "10", "bold"),
            command=self.start_button_command,
        )
        if self.isAnyPortAvailable == False:
            self.connectButton.configure(
                state="disabled", bg="#bbbbbb", highlightbackground="#aaaaaa"
            )

        self.textBox = scrolledtext.ScrolledText(
            self.topFrame,
            bg="#222222",
            fg="#eeeeee",
            border=0,
            wrap="none",
            highlightbackground="#aaaaaa",
            highlightthickness=2,
            font=("Sans", "10", "bold"),
        )

        # Start updating textbox in GUI
        self.recursive_update_textbox()

        ###############################
        ## Widgets size and position ##
        ###############################

        spacing = 10
        padding = 10
        widget_width = 800
        window_width = widget_width + 2 * padding
        window_height = 500

        # Size of application window
        self.window.geometry("{}x{}".format(window_width, window_height))
        # Don't allow resizing in the x or y direction
        self.window.resizable(False, False)

        self.topFrame.configure(padx=padding, pady=padding)
        self.topFrame.place(x=0, y=0, width=window_width, height=window_height)

        self.scanButton.configure(width=widget_width, padx=padding, pady=padding)
        self.scanButton.pack(pady=(0, spacing))

        self.portsOptionMenu.configure(width=widget_width, padx=padding, pady=padding)
        self.portsOptionMenu.pack(pady=(0, spacing))

        self.baudRatesOptionMenu.configure(width=widget_width, padx=padding, pady=padding)
        self.baudRatesOptionMenu.pack(pady=(0, spacing))

        self.connectButton.configure(width=widget_width, padx=padding, pady=padding)
        self.connectButton.pack(pady=(0, spacing))

        self.textBox.configure(width=widget_width, padx=padding, pady=padding)
        self.textBox.pack()

        self.window.protocol("WM_DELETE_WINDOW", self.close_window)
        # Blocking loop for GUI (Always put at the end)
        self.window.mainloop()

    def start_button_command(self):

        if self.isStarted == False:
            self.isStarted = True
            self.connectButton.configure(
                bg="#ba0020",
                highlightbackground="#ffffff",
                activebackground="#cf324d",
                text="Disconnect",
            )
            # Get desired serial port name
            self.serialPortName = self.selectedPort.get()
            # Get desired serial port baud rate
            self.serialPortBaud = self.selectedBaudRate.get()
            # Start Serial Port Communication
            self.serialPortManager.set_name(self.serialPortName)
            self.serialPortManager.set_baud(self.serialPortBaud)
            self.serialPortManager.start()
            # Start updating textbox in GUI
            self.recursive_update_textbox()

        else:
            self.isStarted = False
            self.connectButton.configure(
                bg="#00a832",
                highlightbackground="#ffffff",
                activebackground="#3fcc69",
                text="Connect",
            )
            self.serialPortManager.stop()

    def scan_button_command(self):
        self.portNamesList = self.get_available_serial_ports()

        if len(self.portNamesList) == 0:
            self.isAnyPortAvailable = False
            self.portNamesList = ["No ports available"]
            self.portsOptionMenu.configure(state="disabled")
            self.baudRatesOptionMenu.configure(state="disabled")
            self.connectButton.configure(
                state="disabled", bg="#bbbbbb", highlightbackground="#aaaaaa"
            )
        else:
            self.isAnyPortAvailable = True
            self.portsOptionMenu.configure(state="normal")
            self.baudRatesOptionMenu.configure(state="normal")
            if self.isStarted:
                self.connectButton.configure(
                    bg="#ba0020",
                    highlightbackground="#ffffff",
                    activebackground="#cf324d",
                    state="normal",
                )
            else:
                self.connectButton.configure(
                    bg="#00a832",
                    highlightbackground="#ffffff",
                    activebackground="#3fcc69",
                    state="normal",
                )

        self.update_option_menu(self.portNamesList)

    def get_available_serial_ports(self):
        # Clear portNames list
        portNames = []
        # Get a list of available serial ports
        portsList = list_ports.comports()
        # Sort based on port names
        portsList = sorted(portsList)

        for port in portsList:
            portNames.append(port.device)

        return portNames

    def update_option_menu(self, portNames):
        # Remove old items
        self.portsOptionMenu["menu"].delete(0, "end")
        # Add new items
        for portName in portNames:
            self.portsOptionMenu["menu"].add_command(
                label=portName, command=tk._setit(self.selectedPort, portName)
            )
        # Set default value of selectedPort
        self.selectedPort.set(portNames[0])

    def recursive_update_textbox(self):
        serialPortBuffer = self.serialPortManager.read_buffer()
        # Update textbox in a kind of recursive function using Tkinter after() method
        self.textBox.insert(tk.INSERT, serialPortBuffer.decode("ascii"))
        # autoscroll to the bottom
        self.textBox.see(tk.END)
        # Recursively call recursive_update_textbox using Tkinter after() method
        if self.serialPortManager.isRunning:
            self.window.after(self.guiUpdateInterval, self.recursive_update_textbox)

    def close_window(self):
        if self.isStarted:
            self.serialPortManager.stop()
        self.window.destroy()


class SerialPortManager:
    # A class for management of serial port data in a separate thread
    def __init__(self, serialPortBaud=9600):
        self.isRunning = False
        self.serialPortName = None
        self.serialPortBaud = serialPortBaud
        self.serialPort = serial.Serial()
        # Create a byte array to store incoming data
        self.serialPortBuffer = bytearray()

    def set_name(self, serialPortName):
        self.serialPortName = serialPortName

    def set_baud(self, serialPortBaud):
        self.serialPortBaud = serialPortBaud

    def start(self):
        self.isRunning = True
        self.serialPortThread = threading.Thread(target=self.thread_handler)
        self.serialPortThread.start()

    def stop(self):
        self.isRunning = False

    def thread_handler(self):

        while self.isRunning:

            if not self.serialPort.isOpen():

                self.serialPort = serial.Serial(
                    port=self.serialPortName,
                    baudrate=self.serialPortBaud,
                    bytesize=8,
                    timeout=2,
                    stopbits=serial.STOPBITS_ONE,
                )
            else:
                # Wait until there is data waiting in the serial buffer
                while self.serialPort.in_waiting > 0:
                    # Read only one byte from serial port
                    serialPortByte = self.serialPort.read(1)
                    self.serialPortBuffer.append(int.from_bytes(serialPortByte, byteorder='big'))
                    # Process incoming bytes
                    self.main_process(serialPortByte)

        if self.serialPort.isOpen():
            self.serialPort.close()

    def read_buffer(self):
        # Return a copy of serial port buffer
        buffer = self.serialPortBuffer
        # Clear serial port buffer
        self.serialPortBuffer = bytearray()
        return buffer

    def __del__(self):
        if self.serialPort.isOpen():
            self.serialPort.close()

    def main_process(self, inputByte):
        # Print the received byte in Python terminal
        try:
            character = inputByte.decode("ascii")
        except UnicodeDecodeError:
            pass
        else:
            print(character, end="")


if __name__ == "__main__":

    # Create the GUI
    gui = GUI("Serial Port + Tkinter GUI")

In [ ]:
## This is another example of how to use tkinter to create a GUI for serial communication

import tkinter as tk
from tkinter import ttk
import serial
from threading import Thread
import time

class SerialApp:

    def __init__(self, root):
        self.root = root
        self.root.title("Serial Communication")

        self.ser = serial.Serial('COM7', 115200, timeout=1)
        self.ser.flush()

        self.send_frame = ttk.LabelFrame(root, text="Send")
        self.send_frame.pack(padx=10, pady=10, fill=tk.X)

        self.txt_send = ttk.Entry(self.send_frame, width=40)
        self.txt_send.grid(row=0, column=0, padx=10, pady=5)
        self.txt_send.bind('<Return>', self.send_data)

        self.btn_send = ttk.Button(self.send_frame, text="Send", command=self.send_data)
        self.btn_send.grid(row=0, column=1, padx=10, pady=5)

        self.receive_frame = ttk.LabelFrame(root, text="Receive")
        self.receive_frame.pack(padx=10, pady=10, fill=tk.X, expand=True)

        self.txt_receive = tk.Text(self.receive_frame, height=15, width=50)
        self.txt_receive.grid(row=0, column=0, padx=10, pady=5)

        self.running = True
        self.thread = Thread(target=self.receive_data)
        self.thread.start()

        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def send_data(self, event=None):
        data = self.txt_send.get()
        self.ser.write(data.encode('utf-8'))
        self.txt_send.delete(0, tk.END)

    def receive_data(self):
        while self.running:
            if self.ser.in_waiting:
                data_line = self.ser.readline().decode('utf-8').rstrip()
                self.txt_receive.insert(tk.END, data_line + "\n")
                self.txt_receive.see(tk.END)

    def on_closing(self):
        self.running = False
        self.thread.join()
        self.ser.close()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = SerialApp(root)
    root.mainloop()

In [ ]:
## Serial comm example from chatgpt

# Importing Libraries
import tkinter as tk
import serial
import threading

arduino = serial.Serial(port='COM7', baudrate=115200, timeout=.1)

def write_read(x):
    arduino.write(bytes(x, 'utf-8'))
    time.sleep(0.05)
    data = arduino.readline()
    return data

def send_string():
    write_read("Hello")

def read_string():
    value = write_read("")
    print(value)

# Create the master object
root = tk.Tk()

# Create two buttons
send_button = tk.Button(root, text="Send", command=send_string)
read_button = tk.Button(root, text="Read", command=read_string)

# Place the buttons on the window
send_button.pack()
read_button.pack()

# Run the main loop
root.mainloop()

In [ ]:
## Another serial comm example from chatgpt

import tkinter as tk
import serial

root = tk.Tk()
root.geometry("500x500")
ser = serial.Serial('COM7', 115200)

def foo(x,caller):
    ser.write(bytes(x, 'utf-8'))
    def bar():
        data = ser.readline()
        if caller == 'x': # If called by x
            xvalue.set(data) # Then write to x
        else:  
            yvalue.set(data) # Else write to y
    root.after(1,bar) # Same as 0.001s 

xv = '1'
yv = '2'

xvalue = tk.StringVar()
yvalue = tk.StringVar()

w = tk.Label(root, text="X").place(x=10, y=10)
w1 = tk.Label(root, text="Y").place(x=10, y=40)

display1 = tk.Entry(root, font=("Courier", 16), justify='right', textvariable=xvalue).place(x=50, y=10)
display2 = tk.Entry(root, font=("Courier", 16), justify='right', textvariable=yvalue).place(x=50, y=40)

# Call functions initially
foo(xv,'x')
foo(yv,'y')

root.mainloop()

In [ ]:
import tkinter as tk
import serial  # Serial imported for Serial communication
import threading

# Create the master object
root = tk.Tk()

# Create Serial port object called arduinoSerialData
ArduinoSerial = serial.Serial('COM7', 115200)


def arduino_handler():
    while True:
        data = ArduinoSerial.readline().strip()
        if data.startswith("Temperature:"):
            temperature.set(data.split(":")[1])
        elif data.startswith("Humidity:"):
            humidity.set(data.split(":")[1])


tk.Label(root, text="Humidity:").grid(row=0, column=0, sticky='w')
humidity = tk.StringVar()
tk.Label(root, textvariable=humidity).grid(row=0, column=1, sticky='w')

tk.Label(root, text="Temperature:").grid(row=1, column=0, sticky='w')
temperature = tk.StringVar()
tk.Label(root, textvariable=temperature).grid(row=1, column=1, sticky='w')

threading.Thread(target=arduino_handler, daemon=True).start()
root.mainloop()

In [ ]:
#!/usr/bin/python3
# -*-coding:Utf-8 -*

import sys
import os
import time
import platform
from random import randint
import serial
import serial.tools.list_ports

# interface import
import PySide2


from PySide2.QtWidgets import QApplication, QMainWindow, QDesktopWidget, QTextEdit, QLineEdit, QPushButton, QMessageBox, QWidget, QGridLayout, QTextEdit, QGroupBox, QVBoxLayout, QHBoxLayout, QComboBox, QLabel

from PySide2.QtGui import QIcon, QScreen


__prgm__ = 'Serial Monitor'
__version__ = '0.0.2'


def find_USB_device(USB_DEV_NAME=None):
    myports = [tuple(p) for p in list(serial.tools.list_ports.comports())]
    print(myports)
    usb_port_list = [p[0] for p in myports]
    usb_device_list = [p[1] for p in myports]
    print(usb_device_list)

    if USB_DEV_NAME is None:
        return myports
    else:
        USB_DEV_NAME = str(USB_DEV_NAME).replace("'", "").replace("b", "")
        for device in usb_device_list:
            print("{} -> {}".format(USB_DEV_NAME, device))
            print(USB_DEV_NAME in device)
            if USB_DEV_NAME in device:
                print(device)
                usb_id = device[device.index("COM"):device.index("COM")+4]

                print("{} port is {}".format(USB_DEV_NAME, usb_id))
                return usb_id


class GroupClass(QGroupBox):
    def __init__(self, widget, title="Connection Configuration"):
        super().__init__(widget)
        self.widget = widget
        self.title = title
        self.sep = "-"
        self.id = -1
        self.name = ''
        self.portlist = find_USB_device()
        self.items = [p[0] for p in self.portlist]  # ["COM1","COM2"]
        self.serial = None
        # self.motionDict={"POSITION BASED":" Describe motion based on position","VELOCITY BASED":" Describe motion based on velocity", "LOOP":" Describe loop motion", "PINGPONG":" Describe pingpong motion", "INTERACTIF":" Describe interactive motion"}
        self.init()

    def init(self):
        self.setTitle(self.title)

        self.selectlbl = QLabel("Select port:")
        # label
        self.typeBox = QComboBox()
        self.typeBox.addItems(self.items)  # database getMotionType()
        self.typeBox.setCurrentIndex(self.typeBox.count()-1)

        # btn
        button = QPushButton("Connect")
        button.clicked.connect(self.connect)
        # hbox.addWidget(button)
        sendBtn = QPushButton("send")
        sendBtn.clicked.connect(self.sendData)
        # hbox.addWidget(button)

        titlelbl = QLabel("Enter")
        self.title = QLineEdit("")
        desclbl = QLabel("Console")
        self.desc = QTextEdit("")

        # self.add=QPushButton("Ajouter/Modifier")
        # self.add.clicked.connect(self.addItem)
        # self.rem=QPushButton("Supprimer")
        # self.rem.clicked.connect(self.remItem)

        self.fields = QGridLayout()
        self.fields.addWidget(self.selectlbl, 0, 0, 1, 1)
        self.fields.addWidget(self.typeBox, 0, 1, 1, 1)
        self.fields.addWidget(button, 0, 2, 1, 1)

        self.fields.addWidget(titlelbl, 1, 0, 1, 1)
        self.fields.addWidget(self.title, 1, 1, 1, 1)
        self.fields.addWidget(sendBtn, 1, 2, 1, 1)
        self.fields.addWidget(desclbl, 2, 0, 1, 1)
        self.fields.addWidget(self.desc, 3, 1, 1, 1)
        # self.fields.addWidget(self.add,2,2,1,1)
        # self.fields.addWidget(self.rem,3,2,1,1)
        self.setLayout(self.fields)

    def connect(self):

        self.desc.setText("")
        self.desc.setText(">> trying to connect to port %s ..." %
                          self.typeBox.currentText())
        # with serial.Serial(self.typeBox.currentText(), 115200, timeout=1) as self.serial:
        if self.serial is None:
            self.serial = serial.Serial(
                self.typeBox.currentText(), 115200, timeout=1)
            time.sleep(0.05)
            # self.serial.write(b'hello')
            answer = self.readData()
            if answer != "":
                self.desc.setText(self.desc.toPlainText() +
                                  "\n>> Connected!\n"+answer)
        else:
            self.desc.setText(">> {} already Opened!\n".format(
                self.typeBox.currentText()))

    def sendData(self):
        if self.serial.isOpen():
            if self.title.text() != "":
                self.serial.write(self.title.text().encode())
                answer = self.readData()
                if (self.title.text().encode() == "scan"):
                    print("scanning results -> "+answer.find("0x"))
                else:
                    print(answer.find("0x"))
                self.desc.setText(self.desc.toPlainText()+"\n"+answer)

    def readData(self):
        # self.serial.flush() # it is buffering. required to get the data out *now*
        answer = ""
        while self.serial.inWaiting() > 0:  # self.serial.readable() and

            print(self.serial.inWaiting())
            answer += "\n"+str(self.serial.readline()).replace("\\r",
                                                               "").replace("\\n", "").replace("'", "").replace("b", "")
            # print(self.serial.inWaiting())
        # self.desc.setText(self.desc.toPlainText()+"\n"+answer)
        return answer


class SerialInterface(QMainWindow):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.width = 650
        self.height = 350

        self.resize(self.width, self.height)
        # self.setWindowIcon(QIcon('./resources/logo-100.png'))
        self.setWindowTitle(__prgm__)

        # center window on screen
        qr = self.frameGeometry()
        cp = QScreen().availableGeometry().center()
        qr.moveCenter(cp)

        # init layout
        centralwidget = QWidget(self)
        centralLayout = QHBoxLayout(centralwidget)
        self.setCentralWidget(centralwidget)

        # add connect group
        self.connectgrp = GroupClass(self)
        centralLayout.addWidget(self.connectgrp)


if __name__ == "__main__":
    app = QApplication(sys.argv)
    frame = SerialInterface()
    frame.show()
    sys.exit(app.exec_())

In [ ]:
## 1. List USB devices.
import serial.tools.list_ports

def find_USB_device():
    myports = [tuple(p) for p in list(serial.tools.list_ports.comports())]
    print(myports)
    usb_port_list = [p[0] for p in myports]

    return usb_port_list


find_USB_device()

In [ ]:
#!/usr/bin/env python

# Some of this monitor was made possible with help from those at:
# http://shallowsky.com/blog/hardware/ardmonitor.html
# http://code.activestate.com/recipes/134892/


import sys
import threading
import time
import Queue
import serial


class PythonSerialMonitor():
    def __init__(self):
        self.windows = False
        self.unix = False
        self.fd = None
        self.old_settings = None

        try:
            # Windows
            import msvcrt
            self.windows = True
        except ImportError:
            # Unix
            import sys, tty, termios
            self.fd = sys.stdin.fileno()
            self.old_settings = termios.tcgetattr(self.fd)
            tty.setcbreak(self.fd)
            self.unix = True

        self.input_queue = Queue.Queue()
        self.stop_queue = Queue.Queue()
        self.pause_queue = Queue.Queue()

        self.input_thread = threading.Thread(target=self.add_input, args=(self.input_queue,self.stop_queue,self.pause_queue,))
        self.input_thread.daemon = True
        self.input_thread.start()

    def getch(self):
        if self.unix:
            import sys, tty, termios
            try:
                tty.setcbreak(sys.stdin.fileno())
                ch = sys.stdin.read(1)
            finally:
                termios.tcsetattr(self.fd, termios.TCSADRAIN, self.old_settings)
            return ch
        if self.windows:
            import msvcrt
            return msvcrt.getch()

    def cleanUp(self):
        if self.unix:
            import sys, tty, termios
            termios.tcsetattr(self.fd, termios.TCSADRAIN, self.old_settings)

    def add_input(self, input_queue, stop_queue, pause_queue):
        while True:
            input_queue.put(self.getch())
            if not pause_queue.empty():
                if pause_queue.get() == 'pause':
                    while True:
                        if not pause_queue.empty():
                            if pause_queue.get() == 'resume':
                                break
            if not stop_queue.empty():
                if stop_queue.get() == 'stop':
                    break

    def run(self):
        baud = 9600
        baseports = ['/dev/ttyUSB', '/dev/ttyACM', 'COM', '/dev/tty.usbmodem1234']
        self.ser = None

        while not self.ser:
            for baseport in baseports:
                if self.ser:
                    break
                for i in xrange(0, 64):
                    try:
                        port = baseport + str(i)
                        self.ser = serial.Serial(port, baud, timeout=1)
                        print("Monitor: Opened " + port + '\r')
                        break
                    except:
                        self.ser = None
                        pass

            if not self.ser:
                print("Monitor: Couldn't open a serial port.")
                print("Monitor: Press \'enter\' to try again or \'esc\' to exit.")
                while True:
                    if not self.input_queue.empty():
                        keyboardInput = self.input_queue.get()
                        if ord(keyboardInput) == 27:
                            self.stop_queue.put('stop')
                            self.cleanUp()
                            sys.exit(1)
                        else:
                            # Pressing any key other than 'esc' will continue the monitor
                            break

        self.ser.flushInput()

        while True:
            if not self.input_queue.empty():
                keyboardInput = self.input_queue.get()
                print("Keyboard: " + keyboardInput)
                self.ser.write(keyboardInput)
                if ord(keyboardInput) == 27:
                    self.stop_queue.put('stop')
                    self.cleanUp()
                    sys.exit(1)

            # Check for Teensy output:
            try:
                bytesToRead = self.ser.inWaiting() # get the amount of bytes available at the input queue
                if bytesToRead:
                    line = self.ser.read(bytesToRead) # read the bytes
                    print("Teensy: " + line.strip())
            except IOError:
                # Manually raise the error again so it can be caught outside of this method
                raise IOError()


psm = PythonSerialMonitor()

while True:
    try:
            psm.run()
    except serial.SerialException:
        print ("Monitor: Disconnected (Serial exception)")
    except IOError:
        print ("Monitor: Disconnected (I/O Error)")
    except KeyboardInterrupt:
        print ("Monitor: Keyboard Interrupt. Exiting Now...")
        sys.exit(1)

In [ ]:
"""
Serial Monitor

Designed by ZulNs @Gorontalo, 13 April 2021
"""

import tkinter as tk
from tkinter import ttk as ttk
from tkinter import scrolledtext as tkscroll
from tkinter import messagebox as msgbox
import serial
import serial.tools.list_ports as list_ports
import time
import json
import sys

def get_str_of_chr(chr_in_byte):
	cd = ord(chr_in_byte)
	if 0x20 <= cd and cd <= 0x7e or 0xa1 <= cd:
		if cd == 92:
			return '\\\\'
		return chr(cd)
	else:
		if cd == 9:
			return '\t'
		elif cd == 10:
			return '\n'
		elif cd == 13:
			return '\\r'
	return '\\x{:02x}'.format(cd)

def get_hexstr_of_chr(chr_in_byte):
	cd = ord(chr_in_byte)
	st = '{:02X}'.format(cd)
	if cd == 10:
		st += '\n'
	else:
		st += ' '
	return st

def is_hex_digit(a_byte):
	return b'0' <= a_byte and a_byte <= b'9' or b'A' <= a_byte and a_byte <= b'F' or b'a' <= a_byte and a_byte <= b'f'

def is_oct_digit(a_byte):
	return b'0' <= a_byte and a_byte <= b'7'

def decode_esc(str_of_chr):
	sbs = bytes([ord(c) for c in str_of_chr])
	dbs = b''
	err = None
	idx = 0
	while idx < len(sbs):
		by = sbs[idx:idx+1]
		if by == b'\\':
			idx += 1
			by = sbs[idx:idx+1]
			if by == b'\\' or by == b"'" or by == b'"':
				dbs += by
			elif by == b'0' and not is_oct_digit(sbs[idx+1:idx+2]):
				dbs += b'\0'
			elif by == b'a':
				dbs += b'\a'
			elif by == b'b':
				dbs += b'\b'
			elif by == b't':
				dbs += b'\t'
			elif by == b'n':
				dbs += b'\n'
			elif by == b'v':
				dbs += b'\v'
			elif by == b'f':
				dbs += b'\f'
			elif by == b'r':
				dbs += b'\r'
			elif by == b'x':
				if is_hex_digit(sbs[idx+1:idx+2]):
					if is_hex_digit(sbs[idx+2:idx+3]):
						dbs += bytes([int(sbs[idx+1:idx+3], 16)])
						idx += 3
						continue
				err = {'from': idx-1, 'to': idx+3, 'msg': f'Value Error: invalid {str_of_chr[idx-1:idx+3]} escape at position {idx-1}'}
				break
			elif is_oct_digit(by):
				od = 1
				if is_oct_digit(sbs[idx+1:idx+2]):
					od += 1
					if is_oct_digit(sbs[idx+2:idx+3]):
						od += 1
				ov = int(sbs[idx:idx+od], 8)
				if ov > 255:
					od -= 1
					ov >>= 3
				dbs += bytes([ov])
				idx += od
				continue
			else:
				if by:
					ch = chr(ord(by))
					to = idx + 1
				else:
					ch = ''
					to = idx
				err = {'from': idx-1, 'to': to, 'msg': f"Syntax Error: invalid escape sequence '\\{ch}' at position {idx-1}"}
				break
		else:
			dbs += by
		idx += 1
	return dbs, err

def sendCmd(event):
	global sentTexts, sentTextsPtr
	txt = str(txText.get())
	lst = len(sentTexts)
	if txt == '{about}':
		showAbout()
	if txt != '':
		bs, err = decode_esc(txt)
		if err:
			writeConsole(err['msg'] + '\n', 2)
			txText.xview(err['from'])
			txText.selection_range(err['from'], err['to'])
			txText.icursor(err['to'])
			return
		if lst > 0 and sentTexts[lst-1] != txt or lst == 0:
			sentTexts.append(txt)
		sentTextsPtr = len(sentTexts)
		if lineEndingCbo.current() == 1:
			bs += b'\n'
		elif lineEndingCbo.current() == 2:
			bs += b'\r'
		elif lineEndingCbo.current() == 3:
			bs += b'\r\n'
		currentPort.write(bs)
		if showSentTextVar.get():
			if dispHexVar.get():
				txt = ''.join([get_hexstr_of_chr(bytes([i])) for i in bs])
			else:
				txt = ''.join([get_str_of_chr(bytes([i])) for i in bs])
			writeConsole(txt, 1)
		txText.delete(0, tk.END)

def upKeyCmd(event):
	global sentTextsPtr, lastTxText
	if sentTextsPtr == len(sentTexts):
		lastTxText = str(txText.get())
	if sentTextsPtr > 0:
		sentTextsPtr -= 1
		txText.delete(0, tk.END)
		txText.insert(tk.END, sentTexts[sentTextsPtr])

def downKeyCmd(event):
	global sentTextsPtr
	if sentTextsPtr < len(sentTexts):
		sentTextsPtr += 1
		txText.delete(0, tk.END)
		if sentTextsPtr == len(sentTexts):
			txText.insert(tk.END, lastTxText)
		else:
			txText.insert(tk.END, sentTexts[sentTextsPtr])

def changePort(event):
	global portDesc
	if portCbo.get() == currentPort.port:
		return
	disableSending()
	if currentPort.is_open:
		currentPort.close()
		writeConsole(portDesc + ' closed.\n', 2)
	currentPort.port = portCbo.get()
	portDesc = ports[currentPort.port]
	writeConsole('Opening ' + portDesc + '...', 2)
	root.update()
	try:
		currentPort.open()
	except:
		root.title(APP_TITLE)
		portCbo.set('Select port')
		#msgbox.showerror(APP_TITLE, "Couldn't open the {} port.".format(portDesc))
		writeConsole('failed!!!\n', 2)
		currentPort.port = None
	if currentPort.is_open:
		root.title(APP_TITLE + ': ' + ports[currentPort.port])
		enableSending()
		rxPolling()
		writeConsole('done.\n', 2)
		#msgbox.showinfo(APP_TITLE, '{} opened.'.format(portDesc))

def changeBaudrate(event):
	currentPort.baudrate = BAUD_RATES[baudrateCbo.current()]

def clearOutputCmd():
	global isEndByNL, lastUpdatedBy
	rxText.configure(state=tk.NORMAL)
	rxText.delete('1.0', tk.END)
	rxText.configure(state=tk.DISABLED)
	isEndByNL = True
	lastUpdatedBy = 2

def showTxTextMenu(event):
	if txText.selection_present():
		sta=tk.NORMAL
	else:
		sta=tk.DISABLED
	for i in range(2):
		txTextMenu.entryconfigure(i, state=sta)
	try:
		root.clipboard_get()
		txTextMenu.entryconfigure(2, state=tk.NORMAL)
	except:
		txTextMenu.entryconfigure(2, state=tk.DISABLED)
	try:
		txTextMenu.tk_popup(event.x_root, event.y_root)
	finally:
		txTextMenu.grab_release()

def showRxTextMenu(event):
	if len(rxText.tag_ranges(tk.SEL)):
		rxTextMenu.entryconfigure(0, state=tk.NORMAL)
	else:
		rxTextMenu.entryconfigure(0, state=tk.DISABLED)
	if currentPort.isOpen():
		rxTextMenu.entryconfigure(2, state=tk.NORMAL)
	else:
		rxTextMenu.entryconfigure(2, state=tk.DISABLED)
	try:
		rxTextMenu.tk_popup(event.x_root, event.y_root)
	finally:
		rxTextMenu.grab_release()

def writeConsole(txt, upd=0):
	global isEndByNL, lastUpdatedBy
	tm = ''
	ad = ''
	if upd != 2 and showTimestampVar.get():
		tm = time.strftime('%H:%M:%S.{}'.format(repr(time.time()).split('.')[1][:3]))
	if not upd:
		if not lastUpdatedBy and isEndByNL or lastUpdatedBy:
			if showSentTextVar.get() and showTimestampVar.get():
				ad += 'RX_' + tm
			elif showSentTextVar.get():
				ad += 'RX'
			elif showTimestampVar.get():
				ad += tm
			if ad:
				ad += ' >> '
			if not isEndByNL:
				ad = '\n' + ad
	elif upd == 1:
		if lastUpdatedBy == 1 and isEndByNL or lastUpdatedBy != 1:
			if showTimestampVar.get():
				ad = 'TX_' + tm
			else:
				ad = 'TX'
			ad += ' >> '
			if not isEndByNL:
				ad = '\n' + ad
	elif upd == 2:
		if lastUpdatedBy != 2:
			ad = '\n'
			if not isEndByNL:
				ad += '\n'
	else:
		return
	if upd !=2 and lastUpdatedBy == 2:
		ad = '\n' + ad
	ad += txt
	rxText.configure(state=tk.NORMAL)
	rxText.insert(tk.END, ad)
	if autoscrollVar.get() or upd == 2:
		rxText.see(tk.END)
	rxText.configure(state=tk.DISABLED)
	if txt[-1] == '\n':
		isEndByNL = True
	else:
		isEndByNL = False
	lastUpdatedBy = upd

def rxPolling():
	if not currentPort.is_open:
		return
	preset = time.perf_counter_ns()
	try:
		while currentPort.in_waiting > 0 and time.perf_counter_ns()-preset < 2000000: # loop duration about 2ms
			ch = currentPort.read()
			tm = time.strftime('%H:%M:%S.{}'.format(repr(time.time()).split('.')[1][:3]))
			txt = ''
			if dispHexVar.get():
				txt += get_hexstr_of_chr(ch)
			else:
				txt += get_str_of_chr(ch)
			writeConsole(txt)
	except serial.SerialException as se:
		closePort()
		msgbox.showerror(APP_TITLE, "Couldn't access the {} port".format(portDesc))
	root.after(10, rxPolling) # polling in 10ms interval

def listPortsPolling():
	global ports
	ps = {p.name: p.description for p in list_ports.comports()}
	pn = sorted(ps)
	if pn != portCbo['values']:
		portCbo['values'] = pn
		if len(ports) == 0: # if no port before
			portCbo['state'] = 'readonly'
			portCbo.set('Select port')
			enableSending()
		elif len(pn) == 0: # now if no port
			portCbo['state'] = tk.DISABLED
			portCbo.set('No port')
			disableSending()
		ports = ps
	root.after(1000, listPortsPolling) # polling every 1 second

def disableSending():
	sendBtn['state'] = tk.DISABLED
	txText.unbind('<Return>')

def enableSending():
	sendBtn['state'] = tk.NORMAL
	txText.bind('<Return>', sendCmd)

def closePort():
	if currentPort.is_open:
		currentPort.close()
		writeConsole(portDesc + ' closed.\n', 2)
		currentPort.port = None
		disableSending()
		portCbo.set('Select port')
		root.title(APP_TITLE)

def showAbout():
	msgbox.showinfo(APP_TITLE, 'Designed by ZulNs\n@Gorontalo, 13 April 2021')

def exitRoot():
	data = {}
	data['autoscroll'] = autoscrollVar.get()
	data['showtimestamp'] = showTimestampVar.get()
	data['showsenttext'] = showSentTextVar.get()
	data['displayhex'] = dispHexVar.get()
	data['lineending'] = lineEndingCbo.current()
	data['baudrateindex'] = baudrateCbo.current()
	data['databits'] = currentPort.bytesize
	data['parity'] = currentPort.parity
	data['stopbits'] = currentPort.stopbits
	data['portindex'] = portCbo.current()
	data['portlist'] = ports
	with open(fname+'.json', 'w') as jfile:
		json.dump(data, jfile, indent=4)
		jfile.close()
	root.destroy()

def setting():
	global settingDlg, dataBitsCbo, parityCbo, stopBitsCbo
	settingDlg = tk.Toplevel()
	settingDlg.title('Port Setting')
	if ico:
		settingDlg.iconphoto(False, ico)
	tk.Grid.rowconfigure(settingDlg, 0, weight=1)
	tk.Grid.rowconfigure(settingDlg, 1, weight=1)
	tk.Grid.rowconfigure(settingDlg, 2, weight=1)
	tk.Grid.rowconfigure(settingDlg, 3, weight=1)
	tk.Grid.columnconfigure(settingDlg, 0, weight=1)
	tk.Grid.columnconfigure(settingDlg, 1, weight=1)
	tk.Grid.columnconfigure(settingDlg, 2, weight=1)
	tk.Label(settingDlg, text='Data bits:').grid(row=0, column=1, padx=0, pady=12, sticky=tk.NE)
	tk.Label(settingDlg, text='Parity:').grid(row=1, column=1, padx=0, pady=0, sticky=tk.NS+tk.E)
	tk.Label(settingDlg, text='Stop bits:').grid(row=2, column=1, padx=0, pady=12, sticky=tk.NE)
	dataBitsCbo = ttk.Combobox(settingDlg, width=10, state='readonly')
	dataBitsCbo.grid(row=0, column=2, padx=12, pady=12, sticky=tk.NE)
	dataBitsCbo['values'] = DATABITS
	dataBitsCbo.set(currentPort.bytesize)
	parityCbo = ttk.Combobox(settingDlg, width=10, state='readonly')
	parityCbo.grid(row=1, column=2, padx=12, pady=0, sticky=tk.NS+tk.E)
	parityCbo['values'] = PARITY_VAL
	parityCbo.current(PARITY.index(currentPort.parity))
	stopBitsCbo = ttk.Combobox(settingDlg, width=10, state='readonly')
	stopBitsCbo.grid(row=2, column=2, padx=12, pady=12, sticky=tk.NE)
	stopBitsCbo['values'] = STOPBITS
	stopBitsCbo.set(currentPort.stopbits)
	tk.Button(settingDlg, text='Default', width=10, command=defaultSetting).\
		grid(row=1, column=0, padx=12, pady=0, sticky=tk.NS+tk.W)
	tk.Button(settingDlg, text='OK', width=10, command=lambda:setPort(None)).\
		grid(row=3, column=1, padx=0, pady=12, sticky=tk.S)
	cancelBtn = tk.Button(settingDlg, text='Cancel', width=10, command=lambda:hideSetting(None))
	cancelBtn.grid(row=3, column=2, padx=12, pady=12, sticky=tk.S)
	settingDlg.bind('<Return>', setPort)
	settingDlg.bind('<Escape>', hideSetting)
	settingDlg.update()
	rw = root.winfo_width()
	rh = root.winfo_height()
	rx = root.winfo_rootx()
	ry = root.winfo_rooty()
	dw = settingDlg.winfo_width()
	dh = settingDlg.winfo_height()
	settingDlg.geometry(f'{dw}x{dh}+{rx+int((rw-dw)/2)}+{ry+int((rh-dh)/2)}')
	settingDlg.minsize(dw, dh)
	settingDlg.maxsize(dw, dh)
	settingDlg.resizable(0, 0)
	settingDlg.grab_set()
	cancelBtn.focus_set()

def defaultSetting():
	dataBitsCbo.set(serial.EIGHTBITS)
	parityCbo.current(PARITY.index(serial.PARITY_NONE))
	stopBitsCbo.set(serial.STOPBITS_ONE)

def setPort(event):
	currentPort.bytesize = DATABITS[dataBitsCbo.current()]
	currentPort.parity = PARITY[parityCbo.current()]
	currentPort.stopbits = STOPBITS[stopBitsCbo.current()]
	settingDlg.destroy()

def hideSetting(event):
	settingDlg.destroy()

if __name__ == '__main__':
	APP_TITLE = 'Serial Monitor'
	BAUD_RATES = (300, 1200, 2400, 4800, 9600, 19200, 38400, 57600, 76800, 115200, 23040, 500000, 1000000, 2000000)
	DATABITS = (serial.FIVEBITS, serial.SIXBITS, serial.SEVENBITS, serial.EIGHTBITS)
	PARITY = (serial.PARITY_EVEN, serial.PARITY_ODD, serial.PARITY_NONE, serial.PARITY_MARK, serial.PARITY_SPACE)
	PARITY_VAL = ('Even', 'Odd', 'None', 'Mark', 'Space')
	STOPBITS = (serial.STOPBITS_ONE, serial.STOPBITS_ONE_POINT_FIVE, serial.STOPBITS_TWO)
	ports = {p.name: p.description for p in list_ports.comports()}
	currentPort = serial.Serial(port=None, baudrate=9600, timeout=0, write_timeout=0)
	portDesc = ''
	sentTexts = []
	sentTextsPtr = 0
	isEndByNL = True
	lastUpdatedBy = 2
	ico = None

	data = {}
	fname = sys.argv[0].rsplit('.', 1)[0]
	jfile = None
	try:
		jfile = open(fname+'.json')
		data = json.load(jfile)
	except FileNotFoundError as fnfe:
		pass
	if jfile:
		jfile.close()

	root = tk.Tk()
	root.title(APP_TITLE)
	try:
		ico = tk.PhotoImage(file = fname+'.png')
	except:
		pass
	if ico:
		root.iconphoto(False, ico)
	root.protocol("WM_DELETE_WINDOW", exitRoot)

	autoscrollVar = tk.BooleanVar()
	showTimestampVar = tk.BooleanVar()
	showSentTextVar = tk.BooleanVar()
	dispHexVar = tk.BooleanVar()

	tk.Grid.rowconfigure(root, 0, weight=1)
	tk.Grid.rowconfigure(root, 1, weight=999)
	tk.Grid.rowconfigure(root, 2, weight=1)

	tk.Grid.columnconfigure(root, 0, weight=1)
	tk.Grid.columnconfigure(root, 1, weight=1)
	tk.Grid.columnconfigure(root, 2, weight=1)
	tk.Grid.columnconfigure(root, 3, weight=999)
	tk.Grid.columnconfigure(root, 4, weight=1)
	tk.Grid.columnconfigure(root, 5, weight=1)
	tk.Grid.columnconfigure(root, 6, weight=1)

	txText = tk.Entry(root)
	txText.grid(row=0, column=0, columnspan=6, padx=4, pady=8, sticky=tk.N+tk.EW)
	txText.bind('<Up>', upKeyCmd)
	txText.bind('<Down>', downKeyCmd)
	txText.bind('<Button-3>', showTxTextMenu)

	sendBtn = tk.Button(root, width=12, text='Send', state=tk.DISABLED, command=lambda:sendCmd(None))
	sendBtn.grid(row=0, column=6, padx=4, pady=4, sticky=tk.NE)

	rxText = tkscroll.ScrolledText(root, height=20, state=tk.DISABLED, font=('Courier', 10), wrap=tk.WORD)
	rxText.grid(row=1, column=0, columnspan=7, padx=4, sticky=tk.NSEW)
	rxText.bind('<Button-3>', showRxTextMenu)

	autoscrollCbt = tk.Checkbutton(root, text='Autoscroll', variable=autoscrollVar, onvalue=True, offvalue=False)
	autoscrollCbt.grid(row=2, column=0, padx=4, pady=4, sticky=tk.SW)
	di = data.get('autoscroll')
	if di != None:
		autoscrollVar.set(di)

	showTimestampCbt = tk.Checkbutton(root, text='Show timestamp', variable=showTimestampVar, onvalue=True, offvalue=False)
	showTimestampCbt.grid(row=2, column=1, padx=4, pady=4, sticky=tk.SW)
	di = data.get('showtimestamp')
	if di != None:
		showTimestampVar.set(di)

	showSentTextCbt = tk.Checkbutton(root, text='Show sent text', variable=showSentTextVar, onvalue=True, offvalue=False)
	showSentTextCbt.grid(row=2, column=2, padx=4, pady=4, sticky=tk.SW)
	di = data.get('showsenttext')
	if di != None:
		showSentTextVar.set(di)

	portCbo = ttk.Combobox(root, width=10)
	portCbo.grid(row=2, column=3, padx=4, pady=4, sticky=tk.SE)
	portCbo.bind('<<ComboboxSelected>>', changePort)
	portCbo['values'] = sorted(ports)
	if len(ports) > 0:
		portCbo['state'] = 'readonly'
		portCbo.set('Select port')
	else:
		portCbo['state'] = tk.DISABLED
		portCbo.set('No port')

	lineEndingCbo = ttk.Combobox(root, width=14, state='readonly')
	lineEndingCbo.grid(row=2, column=4, padx=4, pady=4, sticky=tk.SE)
	lineEndingCbo['values'] = ('No line ending', 'Newline', 'Carriage return', 'Both CR & NL')
	di = data.get('lineending')
	if di != None:
		lineEndingCbo.current(di)
	else:
		lineEndingCbo.current(0)

	baudrateCbo = ttk.Combobox(root, width=12, state='readonly')
	baudrateCbo.grid(row=2, column=5, padx=4, pady=4, sticky=tk.SE)
	baudrateCbo['values'] = list(str(b) + ' baud' for b in BAUD_RATES)
	baudrateCbo.bind('<<ComboboxSelected>>', changeBaudrate)
	di = data.get('baudrateindex')
	if di != None:
		baudrateCbo.current(di)
		currentPort.baudrate = BAUD_RATES[di]
	else:
		baudrateCbo.current(4) # 9600 baud
		currentPort.baudrate = BAUD_RATES[4]

	clearBtn = tk.Button(root, width=12, text='Clear output', command=clearOutputCmd)
	clearBtn.grid(row=2, column=6, padx=4, pady=4, sticky=tk.SE)

	txTextMenu = tk.Menu(txText, tearoff=0)
	txTextMenu.add_command(label='Cut', accelerator='Ctrl+X', command=lambda:txText.event_generate('<<Cut>>'))
	txTextMenu.add_command(label='Copy', accelerator='Ctrl+C', command=lambda:txText.event_generate('<<Copy>>'))
	txTextMenu.add_command(label='Paste', accelerator='Ctrl+V', command=lambda:txText.event_generate('<<Paste>>'))

	rxTextMenu = tk.Menu(rxText, tearoff=0)
	rxTextMenu.add_command(label='Copy', accelerator='Ctrl+C', command=lambda:rxText.event_generate('<<Copy>>'))
	rxTextMenu.add_separator()
	rxTextMenu.add_command(label='Close active port', command=closePort)
	rxTextMenu.add_separator()
	rxTextMenu.add_checkbutton(label='Display in hexadecimal code', onvalue=True, offvalue=False, variable=dispHexVar)
	rxTextMenu.add_separator()
	rxTextMenu.add_command(label='Port setting', command=setting)
	rxTextMenu.add_separator()
	rxTextMenu.add_command(label='About', command=showAbout)

	listPortsPolling()

	root.update()
	sw = root.winfo_screenwidth()
	sh = root.winfo_screenheight()
	rw = root.winfo_width()
	rh = root.winfo_height()
	root.minsize(rw, 233)
	root.geometry(f'{rw}x{rh}+{int((sw-rw)/2)}+{int((sh-rh)/2)-30}')

	di = data.get('displayhex')
	if di != None:
		dispHexVar.set(di)
	di = data.get('databits')
	if di != None:
		currentPort.bytesize = di
	di = data.get('parity')
	if di != None:
		currentPort.parity = di
	di = data.get('stopbits')
	if di != None:
		currentPort.stopbits = di
	di = data.get('portindex')
	if di != None and di != -1 and data.get('portlist') == ports:
		portCbo.current(di)
		changePort(None)

	root.mainloop()